In [ ]:
import cv2
import numpy as np
import os

# Load the image
image_path = './opal.jpeg'
assert os.path.exists(image_path)

img = cv2.imread(image_path)

In [ ]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Hide axis
plt.show()

In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.convertScaleAbs(gray)
median_blur = cv2.medianBlur(gray, 5)
plt.imshow(cv2.cvtColor(median_blur, cv2.COLOR_BGR2RGB))

edges = cv2.Canny(median_blur, 100, 200)
plt.imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))

In [ ]:
ret, thresh = cv2.threshold(edges, 127, 255, 0)
plt.imshow(cv2.cvtColor(thresh, cv2.COLOR_BGR2RGB))
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img, contours, -1, (0,255,0), 3)

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))